### 使用3$\sigma$法则定上界, 超出界限的点看作异常，转换为前10个点的最值

In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
def source2normal(path, file, result_path):
#     读取叶子元素实际值
    source_data = pd.read_csv(path + file, usecols=[0])
    source_series = [ int(i) for i in source_data.values]
    
#     存放异常点索引
    length = len(source_series)
#     abnormal_points = []
    smoothed_series = source_series[:]
    abnormal_labels = [0]*length
    pred = [0]*length
    
#     计算上界
    miu = np.mean(source_series)
    sigma = np.std(source_series)
    L = 3
    UCL = miu + L * sigma
#     print("miu = %f" % miu)
#     print("sigma = %f" % sigma)
#     print("Upper Control Line = %f" % UCL)

#     依次替换掉超出上界的异常值
    for i in range(len(source_series)):
        if source_series[i] > UCL:
            abnormal_labels[i] = 1
            # 列表切片是左闭右开区间，且倒序的右开区间最小等于0.索引大于10的取前面10个元素的最值，小于10的取前面所有元素的最值。
            if i == 0:
                smoothed_series[i] = min(np.max(smoothed_series[1:11]), int(UCL))
            elif i > 10:
                smoothed_series[i] = np.max(smoothed_series[i-1:i-11:-1])
            else:
                smoothed_series[i] = np.max(smoothed_series[i-1::-1])
                
    normal_set = {}
    normal_set['true'] = source_series
    normal_set['pred'] = pred
    normal_set['smoothed'] = smoothed_series
    normal_set['label'] = abnormal_labels
    df = pd.DataFrame(normal_set)
    csvFile = open(result_path + file, 'w')
    df.to_csv(result_path + file, index=None)


In [3]:
path = 'leavesFirst2Week/'
result_path = 'smoothingLeavesFirst2Week/'
if not os.path.exists(result_path):
    os.makedirs(result_path)
files = os.listdir(path)
for file in files:
    source2normal(path, file, result_path)